In [ ]:
# %%writefile galapagos_run_fwd.py

from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
from datetime import timedelta as delta
from glob import glob
import numpy as np
import xarray as xr

ddir = '/data2/imau/oceanparcels/hydrodynamic_data/NEMO-MEDUSA/ORCA0083-N006/'
ufiles = sorted(glob(ddir+'means/ORCA0083-N06_*d05U.nc'))
vfiles = [u.replace('05U.nc', '05V.nc') for u in ufiles]
meshfile = glob(ddir+'domain/coordinates.nc')

filenames = {'U': {'lon': meshfile, 'lat': meshfile, 'data': ufiles},
             'V': {'lon': meshfile, 'lat': meshfile, 'data': vfiles}}
variables = {'U': 'uo', 'V': 'vo'}
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'time': 'time_counter'}
fieldset = FieldSet.from_nemo(filenames, variables, dimensions)

fieldset.computeTimeChunk(fieldset.U.grid.time[0], 1)
flon = fieldset.U.grid.lon
flat = fieldset.U.grid.lat

landlon = []
landlat = []
xmin =1700
for y in range(1000, 1900, 6):
    line = fieldset.U.data[0, y, xmin:]
    I = np.where(line==0)[0]
    if len(I)>0:
        for i in I:
            if np.all(line[i:i+10]==0):
                landlon.append(flon[y, xmin+i-1])
                landlat.append(flat[y, xmin+i-1])
                break

galapagosmask = np.zeros_like(fieldset.U.data[0, :, :])
galapagos_extent = [-91.8, -89, -1.4, 0.7]
for x in range(2000, 2500):
    for y in range(1300, 1600):
        if (flon[y, x] >= galapagos_extent[0] and flon[y, x] <= galapagos_extent[1] and
            flat[y, x] >= galapagos_extent[2] and flat[y, x] <= galapagos_extent[3]):
            galapagosmask[y, x] = 1
fieldset.add_field(Field('galapagosmask', galapagosmask, grid=fieldset.U.grid, 
                         mesh='spherical', interp_method='nearest'))

def SampleGalapagos(fieldset, particle, time):
    if fieldset.galapagosmask[time, particle.depth, particle.lat, particle.lon] == 1:
        particle.visitedgalapagos = 1

def Age(fieldset, particle, time):
    particle.age = particle.age + math.fabs(particle.dt)
    if particle.age > 2*365*86400:
        particle.delete()

class GalapagosParticle(JITParticle):
    visitedgalapagos = Variable('visitedgalapagos', initial=0.)
    age = Variable('age', initial = 0.)

pset = ParticleSet(fieldset=fieldset, pclass=GalapagosParticle, lon=landlon, lat=landlat, 
                   time=fieldset.U.grid.time[0], repeatdt=delta(days=5))
outfile = pset.ParticleFile(name="/scratch/evansebille/galapagosparticles_americas.nc", outputdt=delta(days=1))

pset.execute(AdvectionRK4+pset.Kernel(SampleGalapagos)+Age, dt=delta(hours=1), output_file=outfile)

In [ ]:
%time outfile.close()

In [ ]:
%%writefile parcelsrun.sh
#/bin/sh
#
# SGE: the job name
#$ -N Galapagosrunfwd
#
# The requested run-time, expressed as (xxxx sec or hh:mm:ss)
#$ -l h_rt=96:00:00
#
# SGE: ouput in the current working dir
#$ -cwd    
#
source ~/miniconda2/bin/activate root
cd /home/staff/sebil001/ParcelsRuns/Galapagos_NEMO
python galapagos_run_fwd.py

In [ ]:
import os
os.system('qsub -V parcelsrun.sh')